**Importing Packages**

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

**Loading Data**

In [4]:
import pandas as pd

# Load the data
# Check if the file exists in the current directory or specify the full file path
file_path = 'Import Export Trade Data Africa.csv'  # If the file is in the same directory as the notebook
# OR
# file_path = '/path/to/your/file/Import Export Trade Data Africa.csv' # Replace with the actual path if the file is located elsewhere

data = pd.read_csv(file_path)

# Preview the data
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DATE                   22 non-null     object 
 1   TYPE                   22 non-null     object 
 2   CPC DESCRIPTION        22 non-null     object 
 3   EXPORTER NAME          22 non-null     object 
 4   IMPORTER NAME          22 non-null     object 
 5   DECLARANT NAME         22 non-null     object 
 6   Origin Country         22 non-null     object 
 7   Destination Country    22 non-null     object 
 8   HS CODE                22 non-null     int64  
 9   HS CODE DESCRIPTION    22 non-null     object 
 10  QUANTITY UOM           22 non-null     object 
 11  QUANTITY               22 non-null     float64
 12  NO OF PACKAGE TYPE     22 non-null     int64  
 13  GROSS WEIGHT           22 non-null     float64
 14  GROSS WEIGHT UOM       22 non-null     object 
 15  NET WEIG

**Load Data to SQLite**

In [5]:
import sqlite3
import pandas as pd

# File path (adjust if necessary)
file_path = 'Import Export Trade Data Africa.csv'

# Load data into a Pandas DataFrame
data = pd.read_csv(file_path)

# Create an SQLite in-memory database
conn = sqlite3.connect(':memory:')
data.to_sql('trade_data', conn, index=False, if_exists='replace')


22

**Initial Queries**

**Explore the Structure**

Checking the first few rows to ensure the data is correctly loaded.

In [6]:
query = "SELECT * FROM trade_data LIMIT 10;"
print(pd.read_sql(query, conn))


         DATE    TYPE                        CPC DESCRIPTION  \
0  01/11/2019  Export  Direct Exports of home produced goods   
1  01/11/2019  Export  Direct Exports of home produced goods   
2  01/11/2019  Export  Direct Exports of home produced goods   
3  01/11/2019  Export  Direct Exports of home produced goods   
4  01/11/2019  Export  Direct Exports of home produced goods   
5  01/11/2019  Export  Direct Exports of home produced goods   
6  01/11/2019  Export  Direct Exports of home produced goods   
7  01/11/2019  Export  Direct Exports of home produced goods   
8  01/11/2019  Export  Direct Exports of home produced goods   
9  01/11/2019  Export  Direct Exports of home produced goods   

                                 EXPORTER NAME  \
0  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWANA   
1  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWANA   
2  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWANA   
3  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWANA   
4  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWAN

**Check for Missing Data**

Identify columns with missing values.

In [7]:
query = """
SELECT
    COUNT(*) AS total_records,
    SUM(CASE WHEN DATE IS NULL THEN 1 ELSE 0 END) AS missing_DATE,
    SUM(CASE WHEN `EXPORTER NAME` IS NULL THEN 1 ELSE 0 END) AS missing_EXPORTER_NAME,
    SUM(CASE WHEN `IMPORTER NAME` IS NULL THEN 1 ELSE 0 END) AS missing_IMPORTER_NAME
FROM trade_data;
"""
print(pd.read_sql(query, conn))


   total_records  missing_DATE  missing_EXPORTER_NAME  missing_IMPORTER_NAME
0             22             0                      0                      0


**Summarize Trade by Exporter**

Finding the total trade value and quantity for each exporter.

In [8]:
query = """
SELECT
    `EXPORTER NAME`,
    SUM(`QUANTITY`) AS total_quantity,
    SUM(`CUSTOMS VALUE BWP`) AS total_trade_value
FROM trade_data
GROUP BY `EXPORTER NAME`
ORDER BY total_trade_value DESC
LIMIT 10;
"""
print(pd.read_sql(query, conn))


                                 EXPORTER NAME  total_quantity  \
0  ALVIN INVESTMENTS,PLOT 2167,KASANE,BOTSWANA           129.5   

   total_trade_value  
0            5799.58  


**Analyzing Trade Trends by Month**

Summarizing trade value and quantity per month.

In [9]:
query = """
SELECT
    `YEAR`,
    `MONTH`,
    SUM(`CUSTOMS VALUE BWP`) AS total_trade_value,
    SUM(`QUANTITY`) AS total_quantity
FROM trade_data
GROUP BY `YEAR`, `MONTH`
ORDER BY `YEAR`, `MONTH`;
"""
print(pd.read_sql(query, conn))


   YEAR     MONTH  total_trade_value  total_quantity
0  2019  November            5799.58           129.5


 **Find Top Products**

Identifing the top HS Code descriptions by trade value.

In [10]:
query = """
SELECT
    `HS CODE DESCRIPTION`,
    SUM(`CUSTOMS VALUE BWP`) AS total_trade_value,
    SUM(`QUANTITY`) AS total_quantity
FROM trade_data
GROUP BY `HS CODE DESCRIPTION`
ORDER BY total_trade_value DESC
LIMIT 10;
"""
print(pd.read_sql(query, conn))


                                 HS CODE DESCRIPTION  total_trade_value  \
0  Wagyu beef ( other cuts with bone in, fresh or...            1400.00   
1                                       Fresh Grapes            1155.00   
2  Ultra high temperature (UHT) or 'long life' mi...             510.00   
3                         Tomatoes, fresh or chilled             357.60   
4  Leeks and other alliaceous vegetables, fresh o...             289.00   
5                                      Apples, fresh             273.20   
6  Mushrooms of the genus Agaricus, fresh or chilled             271.50   
7                           Olives, fresh or chilled             223.85   
8                                Other melons, fresh             187.75   
9   Cabbage lettuce (head lettuce), fresh or chilled             183.75   

   total_quantity  
0             4.0  
1             4.0  
2            18.0  
3             4.0  
4             4.0  
5            11.0  
6             5.0  
7             

**Checking for Duplicate Records**

Identify rows with duplicate data.

In [11]:
query = """
SELECT
    DATE,
    `EXPORTER NAME`,
    `IMPORTER NAME`,
    `HS CODE`,
    COUNT(*) AS duplicate_count
FROM trade_data
GROUP BY DATE, `EXPORTER NAME`, `IMPORTER NAME`, `HS CODE`
HAVING duplicate_count > 1;
"""
print(pd.read_sql(query, conn))


Empty DataFrame
Columns: [DATE, EXPORTER NAME, IMPORTER NAME, HS CODE, duplicate_count]
Index: []
